In [1]:
import pandas as pd
print("Pandas está funcionando, versión:", pd.__version__)





Pandas está funcionando, versión: 2.2.2


In [2]:
import pandas as pd
import os

# Ruta desde notebooks/ hacia los CSV
data_path = "../data/raw/"

# Listar archivos para confirmar que encuentra los CSV
archivos = os.listdir(data_path)
print("Archivos en la carpeta raw:", archivos)

# Cargar y mostrar las primeras filas del archivo holiday_events.csv
holiday_df = pd.read_csv(os.path.join(data_path, "holidays_events.csv"))

# Mostrar las primeras filas
holiday_df.head()



Archivos en la carpeta raw: ['test.csv', 'train.csv', 'transactions.csv', 'oil.csv', 'holidays_events.csv', 'sample_submission.csv', 'stores.csv']


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [3]:
holiday_df["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [4]:
holiday_df["locale"].unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [5]:
holiday_df["locale_name"].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [6]:
holiday_df["transferred"].unique()

array([False,  True])

In [7]:
holiday_df.dtypes

date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

In [8]:
holiday_df["date"] = pd.to_datetime(holiday_df["date"])

fecha_min = holiday_df["date"].min()
fecha_max = holiday_df["date"].max()
 

print(f" RANGO DE FECHAS DE {fecha_min} a {fecha_max}")


 RANGO DE FECHAS DE 2012-03-02 00:00:00 a 2017-12-26 00:00:00


In [9]:
holiday_df["date"] = pd.to_datetime(holiday_df["date"])  # Asegurar que es datetime
holiday_df["year"] = holiday_df["date"].dt.year
holiday_df["month"] = holiday_df["date"].dt.month
holiday_df["day"] = holiday_df["date"].dt.day
holiday_df["day_of_week"] = holiday_df["date"].dt.weekday  # 0 = Lunes, 6 = Domingo
holiday_df["week_of_year"] = holiday_df["date"].dt.isocalendar().week  # Semana del año

# Mostrar primeras filas con las nuevas columnas
holiday_df.head()


,date,type,locale,locale_name,description,transferred,year,month,day,day_of_week,week_of_year
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2,4,9
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012,4,1,6,13
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12,3,15
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14,5,15
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21,5,16


In [10]:
holiday_df.head()

,date,type,locale,locale_name,description,transferred,year,month,day,day_of_week,week_of_year
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2,4,9
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012,4,1,6,13
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12,3,15
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14,5,15
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21,5,16


In [11]:
num_filas_no_holiday = (holiday_df["type"] != "Holiday" ).sum()
print(f"🔍 Número de filas con 'type' distinto a 'Holiday': {num_filas_no_holiday}")


🔍 Número de filas con 'type' distinto a 'Holiday': 129


In [12]:
num_filas_transferred_holiday = ((holiday_df["type"] == "Holiday") & (holiday_df["transferred"] == True)).sum()
print(f"🔢 Número de filas con 'type'='Holiday' y 'transferred'='True': {num_filas_transferred_holiday}")


🔢 Número de filas con 'type'='Holiday' y 'transferred'='True': 12


In [13]:
# Filtrar solo las filas donde transferred es True y ver los valores únicos de type
valores_type_transferred = holiday_df[holiday_df.transferred == True]["type"].unique()

print("🔍 Valores únicos de 'type' cuando 'transferred' es True:")
print(valores_type_transferred)


🔍 Valores únicos de 'type' cuando 'transferred' es True:
['Holiday']


## Como solo hay true en tranferred cuando es holiday, es que esa columna hace referencia a si ha sido traspasado el día festivo a otro, por lo tanto esos valores no nos interesan, ni la cplumna.

1. Filtramos valores
2. Eliminamos columna

In [14]:
# Filtrar solo los registros donde transferred == False (es decir, NO transferidos)
holiday_df_clean = holiday_df[holiday_df.transferred == False]

# Eliminar la columna transferred, ya que ahora es innecesaria
holiday_df_clean = holiday_df_clean.drop("transferred", axis=1)

# Reiniciar el índice tras la limpieza
holiday_df_clean = holiday_df_clean.reset_index(drop=True)

# Mostrar resultado
holiday_df_clean.head()


,date,type,locale,locale_name,description,year,month,day,day_of_week,week_of_year
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,2012,3,2,4,9
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,2012,4,1,6,13
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,2012,4,12,3,15
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,2012,4,14,5,15
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,2012,4,21,5,16


In [15]:
datos_iniciales = holiday_df.shape[0]

datosSinTransferred= holiday_df[holiday_df["transferred"] != False].shape[0]


holiday_df_clean = holiday_df[holiday_df["transferred"] == False]

datos_postFiltrado = holiday_df_clean.shape[0]

print(f"Los datos iniciales son: {datos_iniciales} los que hemos quitado {datosSinTransferred} los que nos quedan {datos_postFiltrado}  ")

Los datos iniciales son: 350 los que hemos quitado 12 los que nos quedan 338  


In [16]:
holiday_df_clean.head()

,date,type,locale,locale_name,description,transferred,year,month,day,day_of_week,week_of_year
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2,4,9
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012,4,1,6,13
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12,3,15
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14,5,15
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21,5,16


In [17]:
print(holiday_df_clean.isnull().sum())



date            0
type            0
locale          0
locale_name     0
description     0
transferred     0
year            0
month           0
day             0
day_of_week     0
week_of_year    0
dtype: int64


In [19]:
holiday_df_clean['locale_name'].duplicated().sum()

314

In [22]:


# Verificar duplicados en la combinación clave
duplicados = holiday_df_clean.duplicated(subset=["date", "type", "locale", "locale_name","description"], keep=False)

# Mostrar filas duplicadas (si existen)
holiday_df_clean[duplicados]


,date,type,locale,locale_name,description,transferred,year,month,day,day_of_week,week_of_year


In [23]:
## HABEMUS CLAVE ÚNICA